In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [8]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

In [10]:
# import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

In [46]:
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)
from llama_index.indices.postprocessor.node import AutoPrevNextNodePostprocessor

In [47]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor.node import PrevNextNodePostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext

In [14]:
# Load the data
with open('/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl', 'rb') as f:
    cl_techdoc = pickle.load(f)

# convert to llama documents
llama_cl_techdoc = []
for doc in cl_techdoc:
    llama_cl_techdoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

# load the ground truths
with open("/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl", "rb") as f:
    ground_truths = pickle.load(f)

df = pd.DataFrame(ground_truths)

In [16]:
# convert it to nodes
parser = SimpleNodeParser()
cl_techdoc_nodes = parser.get_nodes_from_documents(llama_cl_techdoc)
cl_combineddoc_nodes = parser.get_nodes_from_documents(llama_cl_combineddoc)

In [24]:
def retrieve_docs(df, query_engine):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = query_engine.retrieve(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.node.text
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.node.text

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

### Base

In [20]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)


##### techdoc only

In [ ]:
index_base_techdoc = GPTVectorStoreIndex.from_documents(
    llama_cl_techdoc, 
    service_context=service_context
)

query_engine_base_techdoc = index_base_techdoc.as_query_engine()

In [25]:
df1 = df.copy()
for i, gt in df1.iterrows():
    try:
        df1.loc[i, "result"] = query_engine_base_techdoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df1.loc[i, "result"] = "ERROR"

In [26]:
retrieve_docs(df1, query_engine_base_techdoc)

In [28]:
df1.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv")

##### combineddocs

In [29]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

In [30]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df2.loc[i, "result"] = "ERROR"

In [31]:
retrieve_docs(df2, query_engine_base_combineddoc)

In [32]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv")

### LLMReranker

In [33]:
# based on https://medium.com/llamaindex-blog/using-llms-for-retrieval-and-reranking-23cf2d3a14b6
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
reranker = LLMRerank(choice_batch_size=5, top_n=3, service_context=service_context)

##### techdoc

In [34]:
index_llmreranker_techdoc = GPTVectorStoreIndex.from_documents(llama_cl_techdoc, service_context=service_context)

query_engine_llmreranker_techdoc = index_llmreranker_techdoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [35]:
df3 = df.copy()
for i, gt in df3.iterrows():
    try:
        df3.loc[i, "result"] = query_engine_llmreranker_techdoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df3.loc[i, "result"] = "ERROR"

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1a41b5f75d35c8bb9e19a7c96ef923bf in your message.).


In [36]:
retrieve_docs(df3, query_engine_llmreranker_techdoc)

In [37]:
df3.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_llmreranker.csv")

##### combineddoc

In [38]:
index_llmreranker_combineddoc = GPTVectorStoreIndex.from_documents(llama_cl_combineddoc, service_context=service_context)

query_engine_llmreranker_combineddoc = index_llmreranker_combineddoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [40]:
df4 = df.copy()
for i, gt in df4.iterrows():
    try:
        df4.loc[i, "result"] = query_engine_llmreranker_combineddoc.query(gt.question)
    except Exception as e:
        # logger.error(e)
        df4.loc[i, "result"] = "ERROR"

In [41]:
retrieve_docs(df4, query_engine_llmreranker_combineddoc)

In [42]:
df4.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_llmreranker.csv")

### AutoPrevNext

##### techdoc

In [48]:
service_context = ServiceContext.from_defaults()
nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_techdoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_techdoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_techdoc = index_autopn_techdoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [49]:
df5 = df.copy()
for i, gt in df5.iterrows():
    try:
        df5.loc[i, "result"] = query_engine_autopn_techdoc.query(gt.question)
    except Exception as e:
        df5.loc[i, "result"] = "ERROR"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorr

In [50]:
retrieve_docs(df5, query_engine_autopn_techdoc)

In [51]:
df5.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_autopn.csv")

##### combineddoc

In [52]:
service_context = ServiceContext.from_defaults()

nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_combineddoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_combineddoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_combineddoc = index_autopn_combineddoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [53]:
df6 = df.copy()
for i, gt in df6.iterrows():
    try:
        df6.loc[i, "result"] = query_engine_autopn_combineddoc.query(gt.question)
    except Exception as e:
        df6.loc[i, "result"] = "ERROR"

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorr

In [ ]:
retrieve_docs(df6, query_engine_autopn_combineddoc)

In [ ]:
df6.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_autopn.csv")